In [1]:
import geopandas as gpd 
import numpy as np  
import pandas as pd 
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)

from PIL import Image, ImageOps 
from plotnine import (ggplot, aes, geom_map, geom_text, geom_label, 
                      ggtitle, element_blank, element_rect, 
                      scale_fill_manual, theme_minimal, theme) 
from pulp import (LpProblem, LpMinimize, LpVariable, lpSum, 
                  PULP_CBC_CMD, GLPK_CMD, LpStatus, value) 

In [2]:
# import libraries
import numpy as np
import geopandas as gpd   
import pandas as pd 
from math import pi, pow, sin, cos, asin, sqrt, floor
from pulp import *

In [3]:
def degrees_to_radians(x):
     return((pi / 180) * x)

In [4]:
def lon_lat_distance_miles(lon_a, lat_a, lon_b, lat_b):
    radius_of_earth = 24872 / (2 * pi)
    c = sin((degrees_to_radians(lat_a) - \
    degrees_to_radians(lat_b)) / 2)**2 + \
    cos(degrees_to_radians(lat_a)) * \
    cos(degrees_to_radians(lat_b)) * \
    sin((degrees_to_radians(lon_a) - \
    degrees_to_radians(lon_b))/2)**2
    return(2 * radius_of_earth * (asin(sqrt(c)))) 

In [5]:
def lon_lat_distance_meters (lon_a, lat_a, lon_b, lat_b):
    return(lon_lat_distance_miles(lon_a, lat_a, lon_b, lat_b) * 1609.34)


In [6]:
# read in file with county id, county names, latitudes, longitudes, and populations
file_path = '/Users/Jai/Documents/Git_remote/Decision_analytics/Module6/michigan_counties.xlsx'
michigan_counties = pd.read_excel(file_path, index_col = None)


In [7]:
michigan_counties.head()

,count_id,county_names,latitude,longitude,pop2020
0,0,Leelanau,45.151771,-86.038496,22870
1,1,Clinton,42.943652,-84.601517,79748
2,2,Wexford,44.338367,-85.578414,34196
3,3,Branch,41.916119,-85.059044,44531
4,4,Ionia,42.945094,-85.074603,66809


In [8]:
michigan_counties.shape

(83, 5)

In [9]:
# remove population to allow easy joining of long and lat for each county pair
lat_lon = ['county_names', 'latitude', 'longitude']
lat_lon = michigan_counties[lat_lon]


In [10]:
lat_lon

,county_names,latitude,longitude
0,Leelanau,45.151771,-86.038496
1,Clinton,42.943652,-84.601517
2,Wexford,44.338367,-85.578414
3,Branch,41.916119,-85.059044
4,Ionia,42.945094,-85.074603
...,...,...,...
78,Lapeer,43.090147,-83.221784
79,Arenac,44.042885,-83.747242
80,Charlevoix,45.502498,-85.373250
81,Alcona,44.683623,-83.129008


In [11]:
# create list of county names for pairing        
county_names = michigan_counties['county_names'].to_numpy()


In [12]:
county_names

array(['Leelanau', 'Clinton', 'Wexford', 'Branch', 'Ionia', 'Mecosta',
       'Keweenaw', 'Isabella', 'Schoolcraft', 'Crawford', 'St. Clair',
       'Missaukee', 'Presque Isle', 'Saginaw', 'Houghton', 'Van Buren',
       'Ottawa', 'Berrien', 'Montmorency', 'Shiawassee', 'Otsego',
       'Lenawee', 'Newaygo', 'Roscommon', 'Marquette', 'Alger', 'Iron',
       'Barry', 'Emmet', 'Osceola', 'Antrim', 'Jackson', 'Manistee',
       'Calhoun', 'Tuscola', 'Gladwin', 'Menominee', 'Ontonagon',
       'Gogebic', 'Macomb', 'Midland', 'Kent', 'St. Joseph', 'Ogemaw',
       'Oceana', 'Iosco', 'Alpena', 'Sanilac', 'Oscoda', 'Washtenaw',
       'Kalamazoo', 'Ingham', 'Dickinson', 'Bay', 'Benzie', 'Huron',
       'Clare', 'Luce', 'Genesee', 'Montcalm', 'Cheboygan', 'Eaton',
       'Chippewa', 'Lake', 'Kalkaska', 'Mason', 'Mackinac', 'Oakland',
       'Monroe', 'Allegan', 'Wayne', 'Muskegon', 'Gratiot',
       'Grand Traverse', 'Baraga', 'Delta', 'Hillsdale', 'Cass', 'Lapeer',
       'Arenac', 'Charlevoi

In [13]:
# read in shapefile
file_path = '/Users/Jai/Documents/Git_remote/Decision_analytics/Module6/michigan_counties.geojson'
shapefile_michigan = gpd.read_file(file_path)
map_population_by_county_data = shapefile_michigan.merge(michigan_counties, left_on = 'name', right_on = 'county_names', suffixes = ('_left', '_right'))

# drop unwanted columns
drop_cols = ['statefp', 'countyfp', 'countyns', 'namelsad', 'lsad', 'csafp', 'classfp', 'metdivfp', 'mtfcc', 'cbsafp', 'state_name', 'countyfp_nozero', 'count_id', 'county_names', 'aland', 'awater', 'funcstat']
map_population_by_county_data = map_population_by_county_data.drop(columns = drop_cols)

# check population df; believe that 'geometry' is what's used to create the shape of the state in gpd
map_population_by_county_data.head()


,geo_point_2d,geoid,name,stusab,intptlat,intptlon,geometry,latitude,longitude,pop2020
0,"{'lon': -86.0384960523, 'lat': 45.151770859}",26089,Leelanau,MI,+45.1461816,-086.0515740,"POLYGON ((-85.56175 44.95226, -85.56209 44.950...",45.151771,-86.038496,22870
1,"{'lon': -84.6015165533, 'lat': 42.9436523662}",26037,Clinton,MI,+42.9504550,-084.5916949,"POLYGON ((-84.83762 43.03264, -84.83754 43.032...",42.943652,-84.601517,79748
2,"{'lon': -85.5784138137, 'lat': 44.3383668115}",26165,Wexford,MI,+44.3313751,-085.5700462,"POLYGON ((-85.81909 44.42450, -85.81910 44.425...",44.338367,-85.578414,34196
3,"{'lon': -85.0590443604, 'lat': 41.9161186535}",26023,Branch,MI,+41.9184551,-085.0668852,"POLYGON ((-85.29293 41.98482, -85.29293 41.984...",41.916119,-85.059044,44531
4,"{'lon': -85.0746031181, 'lat': 42.9450938315}",26067,Ionia,MI,+42.9446503,-085.0737660,"POLYGON ((-85.07503 43.12021, -85.06470 43.120...",42.945094,-85.074603,66809


In [14]:
county_populations = np.array(map_population_by_county_data['pop2020'])

In [15]:
county_populations

array([  22870,   79748,   34196,   44531,   66809,   40720,    2180,
         64447,    8188,   13491,  160151,   15213,   13361,  188330,
         37035,   75692,  300873,  152900,    9569,   68022,   25644,
         98567,   50886,   23708,   66661,    8807,   11622,   63554,
         34163,   23274,   24249,  160066,   25287,  133289,   52945,
         25728,   23266,    5863,   14319,  874195,   83674,  659083,
         60874,   20970,   26973,   25521,   28847,   40657,    8404,
        366376,  261173,  284108,   25874,  102821,   18297,   31248,
         31352,    5330,  401983,   67433,   25940,  108992,   36293,
         12594,   18182,   29409,   10941, 1269431,  155609,  121210,
       1757043,  176565,   41100,   96464,    8277,   36741,   45762,
         51403,   88780,   15089,   26293,   10417,  196161])

In [16]:

# model variables
n_counties = 83
n_districts = 14

#n_counties = michigan_counties['county_names'].nunique()

#n_districts = districts['district_name'].nunique()


Objective function and constraints method 1 - Does not work

In [ ]:
#Method 1

model = LpProblem('Compacted-Redistricting', LpMinimize) 


variable_names = [str(i)+str(j) for j in range(1, n_districts+1) \
                                for i in range(1, n_counties+1)]
variable_names.sort() 

DV_variable_y = LpVariable.matrix("Y", variable_names, cat="Binary")
assignment = np.array(DV_variable_y).reshape(83,14)


objective_function = lpSum(assignment) 
model += objective_function

# Solve the model
# !glpsol --help # for details on solver parameters
model.solve()

print('The model status is: ',LpStatus[model.status])
print('The objective value is: ', value(objective_function))

# Initial Assignment / Allocation Constraints

# Allocate 100% of the population from each county.
#for i in range(n_counties):
   # model += lpSum(allocation[i][j] for j in range(n_districts)) == county_populations[i] , "Allocate All " + str(i)


Objective function and constraints method 2 - Does not work

In [ ]:
#Method 2
model = LpProblem('Compacted-Redistricting', LpMinimize) 

# Decision variable: x[i][j] is 1 if county i is assigned to district j, 0 otherwise
x = [[LpVariable(f"x_{i}_{j}", cat="Binary") for j in range(n_districts)] for i in range(n_counties)]

# Objective: Minimize the number of counties that are assigned to multiple districts
model += lpSum(x[i][j] for i in range(n_counties) for j in range(n_districts)) - n_counties

# Constraint: Each county is assigned to exactly one district
for i in range(n_counties):
    model += lpSum(x[i][j] for j in range(n_districts)) == 1


# Allocate 100% of the population from each county.
DV_variable_x = LpVariable.matrix("X", variable_names, cat="Integer", lowBound=0)
allocation = np.array(DV_variable_x).reshape(83,14)  

for i in range(n_counties):
   model += lpSum(allocation[i][j] for j in range(n_districts)) == county_populations[i] , "Allocate All " + str(i)
# Constraint: Each county is assigned to exactly one district
#for i in range(n_counties):
    #model += lpSum(x[i][j] for j in range(n_districts)) == 1

    # Solve the model
# !glpsol --help # for details on solver parameters
model.solve() 
print('The model status is: ',LpStatus[model.status])


# Print the results
for j in range(n_districts):
    district_counties = [i for i in range(n_counties) if x[i][j].varValue == 1]
    print(f"District {j+1} contains counties: {district_counties}")


Objective function and constraints method 3 - Works

In [98]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize

# Assuming n_counties and n_districts are predefined. For example:
n_counties = 83
n_districts = 14

model = LpProblem('Compacted-Redistricting', LpMinimize)

# Decision variable: binary variable for county to district assignment
x = [[LpVariable(f"x_{i}_{j}", cat="Binary") for j in range(n_districts)] for i in range(n_counties)]

# Decision variable: Integer variable for the population allocation from county to district
population_allocation = [[LpVariable(f"pop_{i}_{j}", lowBound=0, cat="Integer") for j in range(n_districts)] for i in range(n_counties)]

# Objective: Minimize the number of counties that are assigned to multiple districts
model += lpSum(x[i][j] for i in range(n_counties) for j in range(n_districts)) - n_counties

# Constraint: Each county is assigned to exactly one district
for i in range(n_counties):
    model += lpSum(x[i][j] for j in range(n_districts)) == 1

# Constraint: Allocate 100% of the population from each county
for i in range(n_counties):
    model += lpSum(population_allocation[i][j] for j in range(n_districts)) == county_populations[i], f"Allocate_All_{i}"

# Ensure population_allocation is consistent with the assignment x
for i in range(n_counties):
    for j in range(n_districts):
        model += population_allocation[i][j] <= county_populations[i] * x[i][j]

#Equal Population Constraint: This ensures each district has roughly the same number of residents.
#ideal_population = sum(county_populations) / n_districts
#tolerance = 0.05  # Example value; typically small.
#for j in range(n_districts):
 #   model += lpSum(population_allocation[i][j] for i in range(n_counties)) <= ideal_population * (1 + tolerance)
 #   model += lpSum(population_allocation[i][j] for i in range(n_counties)) >= ideal_population * (1 - tolerance)

#Respect for Existing Boundaries: It's often preferable to align districts with existing political boundaries, like county or city limits.
for i in range(n_counties):
    for j in range(n_districts):
        model += x[i][j] <= 1  # If a county is split, it should be split into no more than 2 districts.


# Now, you can solve the model (assuming you have a solver like CBC installed)
model.solve()

# And then, you can retrieve the results or the status of the optimization.
print("Status:", LpStatus[model.status])


# Print the results
for j in range(n_districts):
    district_counties = [i for i in range(n_counties) if x[i][j].varValue == 1]
    print(f"District {j+1} contains counties: {district_counties}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Jai/anaconda3/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/mg/gn4s4j_94_j8j560qd6pnnzm0000gp/T/f87d4cc878074bdb9a4a0c07f473e756-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/mg/gn4s4j_94_j8j560qd6pnnzm0000gp/T/f87d4cc878074bdb9a4a0c07f473e756-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2495 COLUMNS
At line 14116 RHS
At line 16607 BOUNDS
At line 18932 ENDATA
Problem MODEL has 2490 rows, 2324 columns and 5810 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 83 - 0.01 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 0 strengthened rows, 238 substitutions
Cgl0004I processed model has 1056 rows, 1848 columns (1848 integer (924 of which binary)) and 3696 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial 